In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install elmoformanylangs

In [0]:
import torch
import os
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.optim

from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from math import ceil
from elmoformanylangs import Embedder

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
classes = ['angry', 'neutral', 'sad', 'happy']
texts = []
labels = []
for idx, label in enumerate(classes):
    PATH = 'drive/My Drive/emotional-classification-hindi/emotions/'+str(label)
    for files in os.walk(PATH):
        for item in files[2]:
            filename = PATH + str('/') + item
            texts.append(open(filename, 'r', encoding='utf-8').read())
            labels.append(idx)

In [6]:
print(texts)
print(labels)

['बार बार एक ही चीज़ बोले जा रहे हो ', 'नहीं डलवाऊँगा पैसे मैं साले चोर ', 'तू गधा है क्या ', 'मेरे ट्रैन का बुकिंग फेल हो रहा है बार बार ', 'अरे ये चैनल कैसे लॉक हो गया गधे ', 'घटिया सर्विस ', 'एक भी सही चीज नहीं बताई तुमने', 'मेरा दिमाग खराब हो गया है इसे रास्ता बताते ', 'आप ऐसे कैसे मेरा पैसा काट सकते हो ', 'एप्प बार बार फस क्यों रहा है ', 'बहुत कन्फुसिंग इंटरफ़ेस है तुम्हारा ', 'मेरे फ्लाइट  का बुकिंग फेल हो रहा है बार बार ', 'मुझे बहुत गुस्सा आ रहा है ', 'यार क्या घटिया सर्विस है ', 'मेरा बैलेंस कैसे ख़तम हो सकता है पागल  ', 'कितनी बार बोला की मेरा डिश टीवी रिचार्ज करना है ', 'क्या बेकार अप्प है ', 'टट्टी एप है ये ', 'लंड जैसा काम किया तुमने ', 'जो बोला वो क्यों नहीं करते हो ? ', 'अरे ये चैनल्स क्यों नहीं आ रहा चूतिये ', 'चुप चाप मेरे पैसे लौटा ', 'यूज़लेस एप ', 'कितनी बार एक ही चीज़ बोलनी पड़ेगी ', 'कुछ भी बोल रहे हो यार ', 'मुझे कुछ समझ में नहीं आ रहा है ', 'गए गुजरे हो तुम हरामजादे', 'गवार एप ', 'अपना पैक अपने पास रख ', 'यार क्या बकवास कर रहे हो तुम ', 'ये कैब कहा रह गया यार ', 'क्य

In [7]:
e = Embedder('drive/My Drive/emotional-classification-hindi/155')

2020-02-19 17:31:01,442 INFO: char embedding size: 4433
2020-02-19 17:31:03,633 INFO: word embedding size: 220391
2020-02-19 17:31:17,486 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(220391, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(4433, 50, padding_idx=4430)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_fe

In [0]:
sent = []
for item in texts:
    sent.append(item.split(' '))
    
sents = []
for item in sent:
    sents.append(item[:len(item)-1])

In [9]:
print(sents)

[['बार', 'बार', 'एक', 'ही', 'चीज़', 'बोले', 'जा', 'रहे', 'हो'], ['नहीं', 'डलवाऊँगा', 'पैसे', 'मैं', 'साले', 'चोर'], ['तू', 'गधा', 'है', 'क्या'], ['मेरे', 'ट्रैन', 'का', 'बुकिंग', 'फेल', 'हो', 'रहा', 'है', 'बार', 'बार'], ['अरे', 'ये', 'चैनल', 'कैसे', 'लॉक', 'हो', 'गया', 'गधे'], ['घटिया', 'सर्विस'], ['एक', 'भी', 'सही', 'चीज', 'नहीं', 'बताई'], ['मेरा', 'दिमाग', 'खराब', 'हो', 'गया', 'है', 'इसे', 'रास्ता', 'बताते'], ['आप', 'ऐसे', 'कैसे', 'मेरा', 'पैसा', 'काट', 'सकते', 'हो'], ['एप्प', 'बार', 'बार', 'फस', 'क्यों', 'रहा', 'है'], ['बहुत', 'कन्फुसिंग', 'इंटरफ़ेस', 'है', 'तुम्हारा'], ['मेरे', 'फ्लाइट', '', 'का', 'बुकिंग', 'फेल', 'हो', 'रहा', 'है', 'बार', 'बार'], ['मुझे', 'बहुत', 'गुस्सा', 'आ', 'रहा', 'है'], ['यार', 'क्या', 'घटिया', 'सर्विस', 'है'], ['मेरा', 'बैलेंस', 'कैसे', 'ख़तम', 'हो', 'सकता', 'है', 'पागल', ''], ['कितनी', 'बार', 'बोला', 'की', 'मेरा', 'डिश', 'टीवी', 'रिचार्ज', 'करना', 'है'], ['क्या', 'बेकार', 'अप्प', 'है'], ['टट्टी', 'एप', 'है', 'ये'], ['लंड', 'जैसा', 'काम', 'किया', 'तुमने'], ['ज

In [10]:
vec = e.sents2elmo(sents)

2020-02-19 17:31:24,083 INFO: 9 batches, avg len: 7.7


In [11]:
maxim = 0
for item in vec:
    if len(item) >= maxim:
        maxim = len(item)

print(maxim)

minim = maxim
for item in vec:
    if len(item) <= minim:
        minim = (len(item))
print(minim)

16
1


In [0]:
padded = []
for i in range (len(vec)):
    siz = maxim - int(vec[i].shape[0])
    zer = np.zeros((siz, 1024), dtype=float)
    padded.append((np.append(vec[i], zer, axis = 0)))

In [0]:
col_names = ['labels','vector']

df_train = pd.DataFrame({'vector':padded, 'labels':labels},
                        columns=col_names)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, labels, test_size=0.1, shuffle=True)
X_tr = X_train['vector']
X_ts = X_test['vector']
X_tr = list(X_tr)
X_ts = list(X_ts)

In [0]:
def trainData(arr, y, batch = 8, istrain = True):
    class txtData(torch.utils.data.Dataset):
        def __init__(self, arr, y, istrain, transform, device='cuda'):
            self.arr = arr
            self.labels = y
            self.istrain = istrain
            self.transform = transform

        def __len__(self):
            return len(self.labels)

        def __getitem__(self, idx):
            data = self.arr[idx]
            label = self.labels[idx]


            if self.istrain:
                return data, label

    data_transform = transforms.Compose([
                transforms.ToTensor()])
    dataset = txtData(arr, y, istrain=istrain, transform=data_transform)
    dataloader = DataLoader(dataset, batch_size = batch, shuffle=True, num_workers=2)

    return dataloader

In [0]:
train_loader = trainData(X_tr, y_train)
test_loader = trainData(X_ts, y_test, batch = 52)

In [0]:
class mainNet(nn.Module):
    def __init__(self):
        super(mainNet, self).__init__()

        self.lstm = nn.LSTM(input_size=1024, hidden_size=512, num_layers=1, bidirectional=False)
        self.fc = nn.Sequential(
                nn.Linear(512, 4),
                nn.ReLU(inplace=False)
                )
        self.dp1 = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self, main):
        main[main != main] = 0.5
        h0 = torch.zeros(1, main.size(0), 512)
        c0 = torch.zeros(1, main.size(0), 512)
        main = main.permute(1, 0, 2)
        output, (final_hidden_state, final_cell_state) = self.lstm(main, (h0, c0))
        output[output != output] = 0.5
        output = output.permute(1,0, 2)
        hidden = final_hidden_state.squeeze(0)
        attn_weights = torch.bmm(output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)
        x = new_hidden_state = torch.bmm(output.transpose(1,2), soft_attn_weights.unsqueeze(2)).squeeze(2)

        x = x.to(device)
        out = self.fc(x)
        out = self.dp1(out)
        fin_out = self.softmax(out)

        return fin_out


In [19]:
device = torch.device("cpu")
model = mainNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_function = torch.nn.CrossEntropyLoss()

num_epochs = 300
losses = []

trn_accuracy = []
val_accuracy = []

for epoch in range(num_epochs):
    print("epoch: {}/{}".format(epoch, num_epochs))
    train_loss = []
    valid_loss = []
    ep_acc = []
    model.train()

    main_iter = iter(train_loader)
    i = 0
    while (i < len(train_loader)):
        accuracy = 0
        mainiter = main_iter.next()

        main, label = mainiter
        main = main.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        main = torch.Tensor.float(main)

        output = model(main)

        _, idx = torch.max(output, dim=1)
        acc = (idx == label)
        accuracy += acc.sum().item() / acc.size(0)

        loss = loss_function(output, label)

        loss.backward()
        optimizer.step()
        i+=1
        ep_acc.append(accuracy)
        train_loss.append(loss.item())

    print("Training")
    trn_accuracy.append((sum(ep_acc)/len(ep_acc)))
    print(sum(ep_acc)/len(ep_acc))


    model.eval()
    ep_acc = []

    main_iter_v = iter(test_loader)
    i = 0
    while (i < len(test_loader)):
        accuracy = 0
        mainiter = main_iter_v.next()
  
        main, label = mainiter
        main = main.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        main = torch.Tensor.float(main)
        output = model(main)

        _, idx = torch.max(output, dim=1)
        acc = (idx == label)
        accuracy += acc.sum().item() / acc.size(0)


        loss = loss_function(output, label)
        i+=1
        ep_acc.append(accuracy)
        train_loss.append(loss.item())
        valid_loss.append(loss.item())

    losses.append([train_loss, valid_loss])
    print("Validation")
    vl_accr = sum(ep_acc)/len(ep_acc)
    val_accuracy.append(vl_accr)
    print(vl_accr)

    best_accuracy = 0
    if vl_accr >= best_accuracy:
        torch.save(model, 'model-'+str(epoch)+str('_1e-5')+'.pth')
        best_accuracy = (sum(ep_acc)/len(ep_acc))

epoch: 0/300
Training
0.3150862068965517
Validation
0.4423076923076923
epoch: 1/300


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type mainNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Training
0.3482758620689655
Validation
0.4423076923076923
epoch: 2/300
Training
0.3681034482758621
Validation
0.4807692307692308
epoch: 3/300
Training
0.31637931034482764
Validation
0.5
epoch: 4/300
Training
0.36637931034482757
Validation
0.5
epoch: 5/300
Training
0.36379310344827587
Validation
0.5
epoch: 6/300
Training
0.38103448275862073
Validation
0.5192307692307693
epoch: 7/300
Training
0.4103448275862069
Validation
0.5192307692307693
epoch: 8/300
Training
0.3935344827586207
Validation
0.5192307692307693
epoch: 9/300
Training
0.36853448275862066
Validation
0.5
epoch: 10/300
Training
0.43232758620689654
Validation
0.4807692307692308
epoch: 11/300
Training
0.4241379310344828
Validation
0.46153846153846156
epoch: 12/300
Training
0.3926724137931034
Validation
0.5
epoch: 13/300
Training
0.38362068965517243
Validation
0.4807692307692308
epoch: 14/300
Training
0.3995689655172414
Validation
0.5192307692307693
epoch: 15/300
Training
0.4047413793103449
Validation
0.5
epoch: 16/300
Training
0